In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
# Libraries
import os
import pickle

from src.models import train_model, test_model, predict_model

In [32]:
# os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

In [35]:
EXPERIMENT = 'mh17'
MODEL = 'model_A3'
EXP_DIR = f'./models/{EXPERIMENT}/'
# Training
PATH_TO_DATA = f'data/processed/synthetic_data/model_A1/n1000/mutate_e2_m8_80/'
# Models
model_template = f'./models/templates/{MODEL}.pl'

Training

In [36]:
train_model.main(
    experiment = EXPERIMENT, 
    path_to_data = PATH_TO_DATA, 
    model_template = model_template, 
    training = 0.8, 
    max_iter = 3, 
    evidence = ['occupancy', 'activity', 'fold_change'],
    facts = [],
    suffix = '', 
)

Splitting data...
Starting LFI...
problog lfi ./models/templates/model_A3.pl ./models/mh17/evidence.pl -O ./models/mh17/model_trained.pl --normalize -k ddnnf -v -n 3
[INFO] Output level: INFO
[INFO] Number of examples: 800
[INFO] Compile time: 4.180552
[INFO] Weights to learn: [t(0.5)::interaction(e1,p0), t(0.5)::interaction(e2,p0), t(0.5)::interaction(e3,p0), t(0.33,P)::fold_change(P,S,inc), t(0.33,P)::fold_change(P,S,dec), t(0.33,P)::fold_change(P,S,baseline), t(0.33,E)::activity(E,S,inc), t(0.33,E)::activity(E,S,dec), t(0.33,E)::activity(E,S,baseline)]
[INFO] Bodies: [lfi_body(0,t,e1,p0), lfi_body(1,t,e2,p0), lfi_body(2,t,e3,p0), lfi_body(3,t(P),P,S,inc), lfi_body(4,t(P),P,S,dec), lfi_body(5,t(P),P,S,baseline), lfi_body(6,t(E),E,S,inc), lfi_body(7,t(E),E,S,dec), lfi_body(8,t(E),E,S,baseline)]
[INFO] Parents: [lfi_par(0,t,e1,p0), lfi_par(1,t,e2,p0), lfi_par(2,t,e3,p0), lfi_par(3,t(P),P,S,inc), lfi_par(4,t(P),P,S,dec), lfi_par(5,t(P),P,S,baseline), lfi_par(6,t(E),E,S,inc), lfi_par(7,t

Testing

In [20]:
# Testing
model_trained = f'{EXP_DIR}model_trained.pl'
test = ['training', 'testing']
PATH_TO_TRUTH = f'data/processed/synthetic_data/model_A1/n1000/'
results = {}

In [22]:
for test_on in test:
    PATH_TO_TESTING = f'{EXP_DIR}{test_on}/'
    compare_to = [test_on, 'truth']
    for compare in compare_to:
        if compare == 'truth':
            path_to_truth = PATH_TO_TRUTH
        else: 
            path_to_truth = PATH_TO_TESTING

        test_results = test_model.main(
            experiment = EXPERIMENT, 
            path_to_data = PATH_TO_TESTING, 
            path_to_truth = path_to_truth, 
            model_trained = model_trained, 
            targets = ['occupancy', 'modification'], 
            test_on = test_on, compare = compare, 
            mode='inference')
        results[f'{test_on}_vs_{compare}'] = test_results

# reindexing issue: change mode to inference

Inferring data (s1)...
problog ./models/mh17/test_on_training.pl -o ./models/mh17/predictions/inferred_example.txt
Finished inference (s1)...
Inferring data (s1000)...
problog ./models/mh17/test_on_training.pl -o ./models/mh17/predictions/inferred_example.txt
Finished inference (s1000)...
Inferring data (s102)...
problog ./models/mh17/test_on_training.pl -o ./models/mh17/predictions/inferred_example.txt
Finished inference (s102)...
Inferring data (s103)...
problog ./models/mh17/test_on_training.pl -o ./models/mh17/predictions/inferred_example.txt
Finished inference (s103)...
Inferring data (s104)...
problog ./models/mh17/test_on_training.pl -o ./models/mh17/predictions/inferred_example.txt
Finished inference (s104)...
Inferring data (s105)...
problog ./models/mh17/test_on_training.pl -o ./models/mh17/predictions/inferred_example.txt
Finished inference (s105)...
Inferring data (s106)...
problog ./models/mh17/test_on_training.pl -o ./models/mh17/predictions/inferred_example.txt
Finished 

In [8]:
PATH_TO_TESTING = f'{EXP_DIR}testing/'
predictions = predict_model.main(
    experiment = EXPERIMENT, 
    path_to_data = PATH_TO_TESTING, 
    model_trained = model_trained, 
    targets = ['occupancy', 'modification'], 
    mode = 'inference', infer_from = 'facts', 
    suffix = '_testing')
results['predictions'] = predictions

Inferring data (s10)...
problog ./models/mh13/make_predictions.pl -o ./models/mh13/predictions/inferred_example.txt
Finished inference (s10)...
Inferring data (s100)...
problog ./models/mh13/make_predictions.pl -o ./models/mh13/predictions/inferred_example.txt
Finished inference (s100)...
Inferring data (s101)...
problog ./models/mh13/make_predictions.pl -o ./models/mh13/predictions/inferred_example.txt
Finished inference (s101)...
Inferring data (s109)...
problog ./models/mh13/make_predictions.pl -o ./models/mh13/predictions/inferred_example.txt
Finished inference (s109)...
Inferring data (s117)...
problog ./models/mh13/make_predictions.pl -o ./models/mh13/predictions/inferred_example.txt
Finished inference (s117)...
Inferring data (s12)...
problog ./models/mh13/make_predictions.pl -o ./models/mh13/predictions/inferred_example.txt
Finished inference (s12)...
Inferring data (s126)...
problog ./models/mh13/make_predictions.pl -o ./models/mh13/predictions/inferred_example.txt
Finished in

In [9]:
with open(f'{EXP_DIR}results.pickle', 'wb') as file:
    pickle.dump(results, file, protocol=pickle.HIGHEST_PROTOCOL)